In [1]:
import numpy as np

## Q learning (Off-Policy TD Control)

same in "2_MC Control agent.ipynb"

In [2]:
class Env:
    def __init__(self):
        self.grid_width = 5
        self.grid_height = self.grid_width
        self.action_grid = [(-1, 0), (1, 0), (0, -1), (0, 1)]     # U, D, L, R
        self.gtriangle1 = [1, 2]
        self.gtriangle2 = [2, 1]
        self.goal = [2, 2]
        
    def step(self, state, action):
        x, y = state
        
        # get next state by action
        x+= action[0]
        y+= action[1]
        
        if x < 0 :
            x = 0
        elif x > (self.grid_width-1) :
            x = (self.grid_width-1)

        if y < 0 :
            y = 0
        elif y > (self.grid_width-1) :
            y = (self.grid_width-1)
        
        next_state = [x, y]
        
        # reward 
        if next_state == self.gtriangle1 or next_state == self.gtriangle2:
            reward = -1
            done = True
        elif next_state == self.goal:
            reward = 1
            done = True
        else:
            reward = 0
            done = False
        
        return next_state, reward, done
    
    def reset(self):
        return [0, 0]

In [3]:
class Qlearning_agent:
    def __init__(self):
        self.action_grid = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        self.action_text= ['U', 'D', 'L', 'R']
        self.grid_width = 5
        self.grid_height = self.grid_width
        self.Qtable = np.zeros((self.grid_width, self.grid_height, len(self.action_grid)))
        self.e = .1
        self.learning_rate = .01
        self.discount_factor = .95
        self.memory=[]
    
    def get_action(self, state):
        # with prob.ε take random action
        if np.random.randn() <  self.e :
            idx = np.random.choice(len(self.action_grid),1)[0]
        else :
            Qvalues = self.Qtable[tuple(state)]
            maxQ = np.amax(Qvalues)
            tie_Qchecker = np.where(Qvalues==maxQ)[0]
            
            # if tie max value, get random
            if len(tie_Qchecker) > 1:
                idx = np.random.choice(tie_Qchecker, 1)[0]
            else :
                idx = np.argmax(Qvalues)
                
        action = self.action_grid[idx]
        return action    
        
    # using First visit MC    
    def update(self, state, action, reward, next_state):
        action_idx = self.action_grid.index(action)
        '''수정한 부분 '''
        #next_action_idx = self.action_grid.index(next_action)
        current_Q = self.Qtable[tuple(state)][action_idx]
        next_Q = max(self.Qtable[tuple(next_state)])

        #print(next_Q)
        #print("끝")
        updated_Q = current_Q + self.learning_rate*((reward + self.discount_factor*next_Q)-current_Q)
        
        self.Qtable[tuple(state)][action_idx] = updated_Q
        
    def save_actionseq(self, action_sequence, action):
        idx = self.action_grid.index(action)
        action_sequence.append(self.action_text[idx])

In [4]:
if __name__ =='__main__':
    env = Env()
    agent = Qlearning_agent()
    total_episode = 10000
    sr = 0
    
    for episode in range(total_episode):
        action_sequence=[]
        total_reward = 0
        walk = 0
        
        # initial state, action, done
        state = env.reset()
        action = agent.get_action(state)
        done = False
        
        while not done:  
            agent.save_actionseq(action_sequence, action)
            '''수정한 부분 '''
            # next state, action
            next_state, reward, done = env.step(state, action)
            #next_action = agent.get_action(next_state)

            # update Qtable
            agent.update(state, action, reward, next_state)
            
            total_reward += reward
            state = next_state
            action = agent.get_action(state)
            
            if done:
                if episode % 100 == 0:
                    print('finished at', next_state)
                    print('episode :{}, The number of step:{}\n The sequence of action is:\
                          {}\nThe total reward is: {}\n'.format(episode, walk, action_sequence, total_reward))
                if state == env.goal:
                    sr += 1
                break

            
            
            
print('The accuracy :', sr/total_episode*100, '%')

finished at [2, 1]
episode :0, The number of step:0
 The sequence of action is:                          ['U', 'R', 'L', 'R', 'U', 'R', 'R', 'L', 'L', 'D', 'D']
The total reward is: -1

finished at [2, 2]
episode :100, The number of step:0
 The sequence of action is:                          ['R', 'R', 'R', 'D', 'U', 'R', 'L', 'D', 'D', 'L']
The total reward is: 1

finished at [1, 2]
episode :200, The number of step:0
 The sequence of action is:                          ['D', 'U', 'R', 'R', 'U', 'D']
The total reward is: -1

finished at [1, 2]
episode :300, The number of step:0
 The sequence of action is:                          ['R', 'U', 'D', 'U', 'R', 'D']
The total reward is: -1

finished at [2, 2]
episode :400, The number of step:0
 The sequence of action is:                          ['D', 'R', 'U', 'R', 'R', 'D', 'D', 'L']
The total reward is: 1

finished at [2, 2]
episode :500, The number of step:0
 The sequence of action is:                          ['R', 'R', 'R', 'U', 'L', '

In [5]:
agent.Qtable

array([[[ 7.35091891e-01,  6.98337295e-01,  7.35091891e-01,
          7.73780937e-01],
        [ 7.73780937e-01,  7.35091815e-01,  7.35091890e-01,
          8.14506250e-01],
        [ 8.14506239e-01, -9.99999999e-01,  7.73780932e-01,
          8.57375000e-01],
        [ 8.57374983e-01,  9.02500000e-01,  8.14506231e-01,
          8.14505820e-01],
        [ 7.97292677e-01,  8.33133656e-01,  8.57374967e-01,
          7.89569452e-01]],

       [[ 7.35091891e-01,  6.17797229e-01,  6.92739288e-01,
          7.29447815e-01],
        [ 7.73780916e-01, -9.86177299e-01,  6.75959714e-01,
         -9.87624007e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 8.57374459e-01,  9.50000000e-01, -9.99999632e-01,
          8.57368592e-01],
        [ 7.77330774e-01,  8.50240166e-01,  9.02499489e-01,
          8.10270074e-01]],

       [[ 6.85146123e-01,  7.16630965e-02,  2.76190716e-01,
         -6.26535720e-01],
        [ 0.00000000e+00,  0.00000000e+

In [6]:
agent.Qtable[0,1]

array([0.77378094, 0.73509181, 0.73509189, 0.81450625])

In [7]:
agent.Qtable[1,1]

array([ 0.77378092, -0.9861773 ,  0.67595971, -0.98762401])

In [8]:
agent.Qtable[1,0]

array([0.73509189, 0.61779723, 0.69273929, 0.72944782])


1. 설명


> 현재 action을 취하고 있는 policy와 improve 하는 대상의 policy가 같을 경우 on policy라고 한다. 이는 greedy한 방법을 사용하는데 한계가 존재한다. e-greedy 방법을 사용하는 대신 action을 취하는 policy 와 업데이트 하는 polict를 다르게 취급하여 학습할 수 있는데 이를 off-policy 방식이라고 한다. Q-learning은 off-policy TD learning의 대표적인 방법으로써 2의 SARSA를 Q-learning으로 수정하였다. off-policy 방식은 여러가지 장점이 있는데 그 장점중 제일 가장 큰 장점은 exploration을 하면서도 optimal한 policy를 학습할수 있다는 것이다. 이렇게 가능한 이유는 action을 취하는 policy(behavior policy)와 improve하는 policy(target policy)를 다르게 하기 때문인데 behavior policy는 e-greedy , target policy greedy 방식으로 학습하게 된다.


> 코드를 보면 get_action에서 behavior policy 에서 action을 선택하는건 같지만 update 함수가 달라진것을 확인 할 수 있다. SARSA의 식에서 nextQ로 업데이트할때 next_Q를 get_action함수를 통해 next action을 받아와서 업데이트 하는 반면에 Q-learning에서는 optimalQ (max값을 가지는 Q)를 greedy 방식으로 선택 하게 하여 업데이트한다. 즉 next action을 따로 get_action으로 받아오는것이 아닌 다음상태에서 최대가되는 maxQ를 사용하는 것이다. 따라서 코드를 보면 next_action을 따로 만들지도 않고 업데이트에서 사용하지도 않는다.

2. 성능 분석


> 성능의 경우 55%정도로 SARSA의 방법과 크게 차이가 나지 않는다. grid world 특성상 2가지 알고리즘 모두 적용 가능하기 때문이다. 하지만 Cliff walking 같은 예제는 다른결과를 낸다. SARSA의 경우는 exploration을 하다가 cliff 에 빠지면 주변 state 모두 낮은 value function을 가져 결국 cliff 쪽으로 가지 않게 된다. 하지만 Q-learning의 경우 cliff에 빠졌더고 해도 거기에 해당하는 action-value function만 낮아지므로 그 근처의 경로를 사용가능하다. 단지 cliff 쪽으로 가지 않는 action만 학습하게 된다. 따라서 SARSA와 달리 cliff와 가까운 optimal path를 찾을 수 있다.




![](https://drive.google.com/file/d/1bAS4PvVj5R1brKIFlueco6mmFRWJqXAZ/view?usp=sharing)
